### Basic processing of Samsung data from Azure cameras

It includes undistortion of color and depth images and reprojection of depth image to undistrorted color virtual camera.

Parameters to set: `SEQUENCE_PATH` - path for recorded sequence, `OUTPUT_DIR` - where postprocessed data will be located.

**Attention**: calibration file `calib_params.yaml` for every camera should be located in original camera data directories.

In [18]:
import cv2
import imageio
import os
import numpy as np
import scipy.ndimage
import shutil

from tqdm import tqdm

from utils.io_utils import filename_to_seconds
from utils.calib_io_utils import load_azure_params
from utils.cv_utils import undistort_image, reproject_depth

In [21]:
SEQUENCE_PATH = '/home/bandeja/datasets/samsung-spatial/2022-04-16-12-15-47_round0/'
OUTPUT_DIR = '/home/bandeja/datasets/samsung-spatial/processed/'

### List recorded cameras

In [22]:
camera_names = os.listdir(SEQUENCE_PATH)
camera_names

['5s', '3m']

### Prepare infrastructure for postprocessed data

In [23]:
dataset_basedir = os.path.basename(os.path.normpath(SEQUENCE_PATH))
postprocessed_data_dir = os.path.join(OUTPUT_DIR, dataset_basedir)

if os.path.exists(postprocessed_data_dir):
    shutil.rmtree(postprocessed_data_dir)
    
os.mkdir(postprocessed_data_dir)

for camera_name in camera_names:
    os.mkdir(os.path.join(postprocessed_data_dir, camera_name))
    os.mkdir(os.path.join(postprocessed_data_dir, camera_name, 'color'))
    os.mkdir(os.path.join(postprocessed_data_dir, camera_name, 'depth'))
    os.mkdir(os.path.join(postprocessed_data_dir, camera_name, 'depth_dense'))

In [24]:
def smooth_depth(depth_image):
    import numpy as np
    KERNEL_SIZE = 11
    MAX_DEPTH_VAL = 1e10
    
    depth_image[depth_image == 0] = MAX_DEPTH_VAL
    smoothed_depth = scipy.ndimage.minimum_filter(depth_image, KERNEL_SIZE)
    smoothed_depth[smoothed_depth > 30000] = 0
    return smoothed_depth

### Iterate over cameras and do undistortion and reprojection 

In [25]:
for camera_name in camera_names:
    camera_data_path = os.path.join(SEQUENCE_PATH, camera_name)
    azure_calib_path = os.path.join(camera_data_path, 'calib_params.json')

    color_camera, depth_camera, T_depth2color = load_azure_params(azure_calib_path)
    np.save(os.path.join(postprocessed_data_dir, camera_name, 'camera_params.npy'), color_camera.K_undist)

    color_images_dir = os.path.join(camera_data_path, 'color')
    color_image_names = os.listdir(color_images_dir)
    color_image_names.sort()
    postprocessed_color_dir = os.path.join(postprocessed_data_dir, camera_name, 'color')

    for color_image_name in tqdm(color_image_names):
        image = cv2.imread(os.path.join(color_images_dir, color_image_name), -1)
        undistorted_image = undistort_image(image, color_camera)
        new_color_image_name = filename_to_seconds(color_image_name)
        imageio.imwrite(os.path.join(postprocessed_color_dir, new_color_image_name), image)

    depth_images_dir = os.path.join(camera_data_path, 'depth')
    depth_image_names = os.listdir(depth_images_dir)
    depth_image_names.sort()
    postprocessed_depth_dir = os.path.join(postprocessed_data_dir, camera_name, 'depth')
    postprocessed_depth_dense_dir = os.path.join(postprocessed_data_dir, camera_name, 'depth_dense')

    for depth_image_name in tqdm(depth_image_names):
        image = cv2.imread(os.path.join(depth_images_dir, depth_image_name), -1)
        undistorted_image = undistort_image(image, depth_camera)
        reprojected_image = reproject_depth(undistorted_image, depth_camera, color_camera, T_depth2color)
        new_depth_image_name = filename_to_seconds(depth_image_name)
        imageio.imwrite(os.path.join(postprocessed_depth_dir, new_depth_image_name), reprojected_image)
        imageio.imwrite(os.path.join(postprocessed_depth_dense_dir, new_depth_image_name), smooth_depth(reprojected_image))

100%|██████████| 1697/1697 [19:01<00:00,  1.49it/s]
